In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# data_Num is a mumerical dataframe

In [3]:
data_Num = pd.read_csv("numerical.csv")

In [4]:
# removing unwanted columns

In [5]:
data_Num.drop(["Unnamed: 0"],axis =1,inplace = True)

In [6]:
#data_Cat is categorical colum

In [7]:
data_Cat = pd.read_csv("data_Cat.csv")

In [8]:
data_Num.shape

(9134, 9)

In [9]:
data_Cat.drop(["Unnamed: 0","Effective To Date"],axis =1,inplace = True)


In [10]:
data_Cat.shape

(9134, 16)

In [11]:
data_Cat.columns

Index(['state', 'response', 'coverage', 'education', 'employmentstatus',
       'gender', 'location_code', 'marital_status', 'policy_type', 'policy',
       'renew_offer_type', 'sales_channel', 'vehicle_class', 'vehicle_size',
       'mm', 'dd'],
      dtype='object')

# Concatinating num and cat

In [12]:
data = pd.concat([data_Num,data_Cat],axis =1)

In [13]:
data.shape

(9134, 25)

# 1)X/y Split

In [14]:
X = data.drop(['total_claim_amount'],axis=1)
y = data['total_claim_amount']


# Train/Test split

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=100)


# Separate X_train and X_test into numerical and categorical (X_train_cat , X_train_num , X_test_cat , X_test_num)

In [16]:
X_train_cat = X_train.select_dtypes(include = "object")
X_train_num = X_train.select_dtypes(exclude = "object")

In [17]:
X_test_cat = X_test.select_dtypes(include = "object")
X_test_num = X_test.select_dtypes(exclude = "object")

In [18]:
print("shape: X_train_cat: ",X_train_cat.shape)
print("shape: X_train_num: ",X_train_num.shape)
print("shape: X_test_cat: ",X_test_cat.shape)
print("shape: X_test_num : ",X_test_num .shape)

shape: X_train_cat:  (5480, 11)
shape: X_train_num:  (5480, 13)
shape: X_test_cat:  (3654, 11)
shape: X_test_num :  (3654, 13)


# Use X_train_num to fit scalers. Transform BOTH X_train_num and X_test_num.

In [19]:
from sklearn.preprocessing import StandardScaler

transformer = StandardScaler().fit(X_train_num)
numericals_train_standardized = transformer.transform(X_train_num)
numericals_test_standardized = transformer.transform(X_test_num)

# Encode the categorical variables X_train_cat and X_test_cat 

In [20]:
X_train_num.columns

Index(['customer_lifetime_value', 'income', 'monthly_premium_auto',
       'months_since_last_claim', 'months_since_policy_inception',
       'number_of_open_complaints', 'number_of_policies', 'response', 'gender',
       'policy', 'renew_offer_type', 'mm', 'dd'],
      dtype='object')

In [21]:
X_train_cat = X_train_cat.drop(['effective_to_date'],axis=1)
X_test_cat = X_test_cat.drop(['effective_to_date'],axis=1)

In [22]:
X_train_cat.shape

(5480, 10)

In [23]:
#X_train_cat_one_hot = ['state','marital_status','policy_type','sales_channel', 'vehicle_class']

In [24]:
#X_train_cat_ordinal = ['coverage', 'education', 'employmentstatus', 'location_code','vehicle_size']

In [25]:
X_train_CAT_One_hot = X_train_cat[['state','marital_status','policy_type','sales_channel', 'vehicle_class']]
X_train_CAT_ordinal = X_train_cat[['coverage', 'education', 'employmentstatus', 'location_code','vehicle_size']]

In [26]:
X_test_CAT_One_hot = X_test_cat[['state','marital_status','policy_type','sales_channel', 'vehicle_class']]
X_test_CAT_ordinal = X_test_cat[['coverage', 'education', 'employmentstatus', 'location_code','vehicle_size']]

# One hot Encoding

In [28]:
# One hot encoding
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='error', drop='first').fit(X_train_CAT_One_hot)
categoricals_train_encoded = encoder.transform(X_train_CAT_One_hot).toarray()
categoricals_test_encoded = encoder.transform(X_test_CAT_One_hot).toarray()

In [30]:
categoricals_train_encoded.shape 

(5480, 15)

In [31]:
categoricals_test_encoded.shape

(3654, 15)

# Ordinal Encoding

In [32]:
#Ordianl Encoder
from sklearn.preprocessing import OrdinalEncoder
encoder_O = OrdinalEncoder()
categoricals_train_Ord = encoder_O.fit_transform(X_train_CAT_ordinal)
categoricals_test_Ord = encoder_O.transform(X_test_CAT_ordinal)

In [33]:
categoricals_train_Ord.shape 

(5480, 5)

In [36]:
X_Cat_train_Final = np.concatenate([categoricals_train_encoded,categoricals_train_Ord],axis = 1)

In [37]:
X_Cat_test_Final = np.concatenate([categoricals_test_encoded,categoricals_test_Ord],axis = 1)

# Final X_train and X_test

In [38]:
X_train = np.concatenate((numericals_train_standardized,X_Cat_train_Final),axis=1)
X_test = np.concatenate((numericals_test_standardized,X_Cat_test_Final),axis=1)

# Linear Regression Model

In [42]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [41]:
# Create a linear regression model
model = LinearRegression()

# Train the model using the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)


In [43]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("R-squared (R2) Score:", r2)

Mean Absolute Error: 129.80495492080667
Mean Squared Error: 31807.98819609267
Root Mean Squared Error: 178.34794138451016
R-squared (R2) Score: 0.5996623677582102


In [46]:
from sklearn.neighbors import KNeighborsRegressor

# function that takes a list of models and train (and tests)

In [60]:

def choose_model(model,X_train,X_test,y_train,y_test):
    if model == "linear regression":
        model1 = LinearRegression()
        model1.fit(X_train, y_train)
        y_pred = model1.predict(X_test)
        return r2_score(y_test, y_pred)
    else:
        model == "K-Neighbourhood"
        KNN = KNeighborsRegressor(n_neighbors=5,p=1)
        KNN.fit(X_train, y_train)
        return KNN.score(X_test, y_test)

choose_model("K-Neighbourhood",X_train,X_test,y_train,y_test)    


0.6144846932764858

In [56]:
choose_model("linear regression",X_train,X_test,y_train,y_test) 

0.5996623677582102

# MLP Regressor

In [63]:
from sklearn.datasets import make_regression
from sklearn.neural_network import MLPRegressor

# Create an MLPRegressor model
#model = MLPRegressor(hidden_layer_sizes=(100,), activation='relu', solver='adam', max_iter=1000, random_state=42)
model = MLPRegressor()


# Train the model using the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Calculate evaluation metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

C:\Users\Denish\AppData\Roaming\Python\Python38\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [64]:
r2

0.7406915750694076

In [ ]:
#Conclusion
# Linear Regression R2 Score:0.5996623677582102
# K-Neighbourhood R2 Score:0.6144846932764858
# MLPRegressor R2 Score:0.7406915750694076
# As the result says MLP gives the best result with default parameters. we can improve it with changing the parameters
# K-Neighbourhood gives also good result with p =1, which is manhatted distance insted of eulidien distance